Source :https://www.kaggle.com/code/dsxavier/nlp-med-dialogue-analysis-engineering/notebook#(A)-Install-Dependencies

In [ ]:
# Install Dependencies
!pip install -q feather-format
!pip install -q pandarallel==1.4.8
!pip install -q pyspellchecker
!pip install -q contractions
!pip install -q distilbert-punctuator
!pip install -q -U git+https://github.com/PrithivirajDamodaran/Gramformer.git
!python3 -m spacy download en

     |████████████████████████████████| 2.5 MB 6.5 MB/s 
     |████████████████████████████████| 287 kB 5.5 MB/s 
     |████████████████████████████████| 110 kB 54.2 MB/s 
     |████████████████████████████████| 5.8 MB 11.9 MB/s 
     |████████████████████████████████| 13.7 MB 55.6 MB/s 
     |████████████████████████████████| 182 kB 51.9 MB/s 
     |████████████████████████████████| 7.6 MB 27.7 MB/s 
     |████████████████████████████████| 1.3 MB 7.3 MB/s 
     |████████████████████████████████| 499 kB 60.6 MB/s 
     |████████████████████████████████| 2.2 MB 57.3 MB/s 
     |████████████████████████████████| 5.0 MB 39.0 MB/s 
     |████████████████████████████████| 211 kB 78.8 MB/s 
     |████████████████████████████████| 1.1 MB 57.3 MB/s 
     |████████████████████████████████| 174 kB 63.3 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
en-core-web-sm 

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
dataset_path = 'gdrive/My Drive/fyp/cnn_dailymail/cnn_dailymail.csv'

In [ ]:
# Import Libraries
import os
import time
import datetime
import warnings
warnings.filterwarnings('ignore')
import threading
import time, timeit
from typing import Text, Dict, List, Union, Tuple, Optional, Set
import re
from ast import literal_eval
from collections import defaultdict, Counter, OrderedDict

import pandas as pd
pd.set_option('display.max_colwidth', 200)
warnings.filterwarnings('ignore')

from pandarallel import pandarallel # Panadas Asynchronous processing
pandarallel.initialize()

import numpy as np

import seaborn as sns
import matplotlib.pylab as plt

import torch
import nltk.data
import contractions
from spellchecker import SpellChecker
from gramformer import Gramformer

from dbpunctuator.inference import Inference, InferenceArguments
from dbpunctuator.utils import ALL_PUNCS, DEFAULT_ENGLISH_TAG_PUNCTUATOR_MAP
from dbpunctuator.utils.utils import register_logger

from tqdm.auto import trange, tqdm
from IPython.display import clear_output

INFO: Pandarallel will run on 2 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [ ]:
plt.rcParams['figure.figsize'] = [18, 15]
plt.style.use('fivethirtyeight')

In [ ]:
sns.set(style='darkgrid')
sns.set(rc={'figure.figsize':(18, 15)})
sns.set_theme(style="darkgrid")

In [ ]:
pd.set_option('display.max_colwidth', 500)
pd.set_option('display.width', 1000)

In [ ]:
# https://stackoverflow.com/a/20354129/14473118
import multiprocessing
pool = multiprocessing.Pool()
print(f'Number of workers for parallelism: {pool._processes}')

Number of workers for parallelism: 2


In [ ]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [ ]:
df = pd.read_csv(dataset_path)
df.head(2)

,article,highlights
0,"b""By. Associated Press. PUBLISHED:. 14:11 EST, 25 October 2013. |. UPDATED:. 15:36 EST, 25 October 2013. The bishop of the Fargo Catholic Diocese in North Dakota has exposed potentially hundreds of church members in Fargo, Grand Forks and Jamestown to the hepatitis A virus in late September and early October. The state Health Department has issued an advisory of exposure for anyone who attended five churches and took communion. Bishop John Folda (pictured) of the Fargo Catholic Diocese in No...","b'Bishop John Folda, of North Dakota, is taking time off after being diagnosed.\nHe contracted the infection through contaminated food in Italy.\nChurch members in Fargo, Grand Forks and Jamestown could have been exposed.'"
1,"b'(CNN) -- Ralph Mata was an internal affairs lieutenant for the Miami-Dade Police Department, working in the division that investigates allegations of wrongdoing by cops. Outside the office, authorities allege that the 45-year-old longtime officer worked with a drug trafficking organization to help plan a murder plot and get guns. A criminal complaint unsealed in U.S. District Court in New Jersey Tuesday accuses Mata, also known as ""The Milk Man,"" of using his role as a police officer to he...","b'Criminal complaint: Cop used his role to help cocaine traffickers.\nRalph Mata, an internal affairs lieutenant, allegedly helped group get guns.\nHe also arranged to pay two assassins in a murder plot, a complaint alleges.'"


In [ ]:
df = df.rename(columns={'article': 'text'})
df = df.rename(columns={'highlights': 'summary'})

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   text     5000 non-null   object
 1   summary  5000 non-null   object
dtypes: object(2)
memory usage: 78.2+ KB


In [ ]:
# Select 1k of dataset
df = df[200:250]
df_copy = df.copy()

In [ ]:
df.head(3)

,text,summary
0,"b""By. Associated Press. PUBLISHED:. 14:11 EST, 25 October 2013. |. UPDATED:. 15:36 EST, 25 October 2013. The bishop of the Fargo Catholic Diocese in North Dakota has exposed potentially hundreds of church members in Fargo, Grand Forks and Jamestown to the hepatitis A virus in late September and early October. The state Health Department has issued an advisory of exposure for anyone who attended five churches and took communion. Bishop John Folda (pictured) of the Fargo Catholic Diocese in No...","b'Bishop John Folda, of North Dakota, is taking time off after being diagnosed.\nHe contracted the infection through contaminated food in Italy.\nChurch members in Fargo, Grand Forks and Jamestown could have been exposed.'"
1,"b'(CNN) -- Ralph Mata was an internal affairs lieutenant for the Miami-Dade Police Department, working in the division that investigates allegations of wrongdoing by cops. Outside the office, authorities allege that the 45-year-old longtime officer worked with a drug trafficking organization to help plan a murder plot and get guns. A criminal complaint unsealed in U.S. District Court in New Jersey Tuesday accuses Mata, also known as ""The Milk Man,"" of using his role as a police officer to he...","b'Criminal complaint: Cop used his role to help cocaine traffickers.\nRalph Mata, an internal affairs lieutenant, allegedly helped group get guns.\nHe also arranged to pay two assassins in a murder plot, a complaint alleges.'"
2,"b""A drunk driver who killed a young woman in a head-on crash while checking his mobile phone has been jailed for six years. Craig Eccleston-Todd, 27, was driving home from a night at a pub when he received a text message. As he was reading or replying to it, he veered across the road while driving round a bend and smashed into Rachel Titley\xe2\x80\x99s car coming the other way. Craig Eccleston-Todd, 27 (left) was using his mobile phone when he crashed head-on into the car being driven by Ra...","b""Craig Eccleston-Todd, 27, had drunk at least three pints before driving car.\nWas using phone when he veered across road in Yarmouth, Isle of Wight.\nCrashed head-on into 28-year-old Rachel Titley's car, who died in hospital.\nPolice say he would have been over legal drink-drive limit at time of crash.\nHe was found guilty at Portsmouth Crown Court of causing death by dangerous driving."""


Let's apply Feature engineering; first to clean the data and prepare it for modelling, and second to understand more about our data and how we can make it more beneficial for having better results.

(A) Remove Markdown links

In [ ]:
def md_links(text: Text) -> Text:
    markdown_link=re.compile(r'\[.*?\]\(.*?\)')
    return markdown_link.sub(r'',text)

In [ ]:
df['text'] = df['text'].parallel_apply(lambda sentence: md_links(sentence))
df['summary'] = df['summary'].parallel_apply(lambda sentence: md_links(sentence))

In [ ]:
df.head(3)

,text,summary
0,"b""By. Associated Press. PUBLISHED:. 14:11 EST, 25 October 2013. |. UPDATED:. 15:36 EST, 25 October 2013. The bishop of the Fargo Catholic Diocese in North Dakota has exposed potentially hundreds of church members in Fargo, Grand Forks and Jamestown to the hepatitis A virus in late September and early October. The state Health Department has issued an advisory of exposure for anyone who attended five churches and took communion. Bishop John Folda (pictured) of the Fargo Catholic Diocese in No...","b'Bishop John Folda, of North Dakota, is taking time off after being diagnosed.\nHe contracted the infection through contaminated food in Italy.\nChurch members in Fargo, Grand Forks and Jamestown could have been exposed.'"
1,"b'(CNN) -- Ralph Mata was an internal affairs lieutenant for the Miami-Dade Police Department, working in the division that investigates allegations of wrongdoing by cops. Outside the office, authorities allege that the 45-year-old longtime officer worked with a drug trafficking organization to help plan a murder plot and get guns. A criminal complaint unsealed in U.S. District Court in New Jersey Tuesday accuses Mata, also known as ""The Milk Man,"" of using his role as a police officer to he...","b'Criminal complaint: Cop used his role to help cocaine traffickers.\nRalph Mata, an internal affairs lieutenant, allegedly helped group get guns.\nHe also arranged to pay two assassins in a murder plot, a complaint alleges.'"
2,"b""A drunk driver who killed a young woman in a head-on crash while checking his mobile phone has been jailed for six years. Craig Eccleston-Todd, 27, was driving home from a night at a pub when he received a text message. As he was reading or replying to it, he veered across the road while driving round a bend and smashed into Rachel Titley\xe2\x80\x99s car coming the other way. Craig Eccleston-Todd, 27 (left) was using his mobile phone when he crashed head-on into the car being driven by Ra...","b""Craig Eccleston-Todd, 27, had drunk at least three pints before driving car.\nWas using phone when he veered across road in Yarmouth, Isle of Wight.\nCrashed head-on into 28-year-old Rachel Titley's car, who died in hospital.\nPolice say he would have been over legal drink-drive limit at time of crash.\nHe was found guilty at Portsmouth Crown Court of causing death by dangerous driving."""


(B) Remove Hyperlinks

In [ ]:
def scrape_links(text):
    url = re.compile(r'https?://\S+|www\.\S+')
    return url.sub(r'',text)

In [ ]:
df['text'] = df['text'].parallel_apply(lambda sentence: scrape_links(sentence))
df['summary'] = df['summary'].parallel_apply(lambda sentence: scrape_links(sentence))

In [ ]:
df.head(3)

,text,summary
0,"b""By. Associated Press. PUBLISHED:. 14:11 EST, 25 October 2013. |. UPDATED:. 15:36 EST, 25 October 2013. The bishop of the Fargo Catholic Diocese in North Dakota has exposed potentially hundreds of church members in Fargo, Grand Forks and Jamestown to the hepatitis A virus in late September and early October. The state Health Department has issued an advisory of exposure for anyone who attended five churches and took communion. Bishop John Folda (pictured) of the Fargo Catholic Diocese in No...","b'Bishop John Folda, of North Dakota, is taking time off after being diagnosed.\nHe contracted the infection through contaminated food in Italy.\nChurch members in Fargo, Grand Forks and Jamestown could have been exposed.'"
1,"b'(CNN) -- Ralph Mata was an internal affairs lieutenant for the Miami-Dade Police Department, working in the division that investigates allegations of wrongdoing by cops. Outside the office, authorities allege that the 45-year-old longtime officer worked with a drug trafficking organization to help plan a murder plot and get guns. A criminal complaint unsealed in U.S. District Court in New Jersey Tuesday accuses Mata, also known as ""The Milk Man,"" of using his role as a police officer to he...","b'Criminal complaint: Cop used his role to help cocaine traffickers.\nRalph Mata, an internal affairs lieutenant, allegedly helped group get guns.\nHe also arranged to pay two assassins in a murder plot, a complaint alleges.'"
2,"b""A drunk driver who killed a young woman in a head-on crash while checking his mobile phone has been jailed for six years. Craig Eccleston-Todd, 27, was driving home from a night at a pub when he received a text message. As he was reading or replying to it, he veered across the road while driving round a bend and smashed into Rachel Titley\xe2\x80\x99s car coming the other way. Craig Eccleston-Todd, 27 (left) was using his mobile phone when he crashed head-on into the car being driven by Ra...","b""Craig Eccleston-Todd, 27, had drunk at least three pints before driving car.\nWas using phone when he veered across road in Yarmouth, Isle of Wight.\nCrashed head-on into 28-year-old Rachel Titley's car, who died in hospital.\nPolice say he would have been over legal drink-drive limit at time of crash.\nHe was found guilty at Portsmouth Crown Court of causing death by dangerous driving."""


(C) Dealing with Tags

In [ ]:
def remove_html_tags(text: Text) -> Text:
    html=re.compile(r'<.*?>')
    return html.sub(r'',text)

In [ ]:
df['text'] = df['text'].parallel_apply(lambda sentence: remove_html_tags(sentence))
df['summary'] = df['summary'].parallel_apply(lambda sentence: remove_html_tags(sentence))

In [ ]:
df.head(3)

,text,summary
0,"b""By. Associated Press. PUBLISHED:. 14:11 EST, 25 October 2013. |. UPDATED:. 15:36 EST, 25 October 2013. The bishop of the Fargo Catholic Diocese in North Dakota has exposed potentially hundreds of church members in Fargo, Grand Forks and Jamestown to the hepatitis A virus in late September and early October. The state Health Department has issued an advisory of exposure for anyone who attended five churches and took communion. Bishop John Folda (pictured) of the Fargo Catholic Diocese in No...","b'Bishop John Folda, of North Dakota, is taking time off after being diagnosed.\nHe contracted the infection through contaminated food in Italy.\nChurch members in Fargo, Grand Forks and Jamestown could have been exposed.'"
1,"b'(CNN) -- Ralph Mata was an internal affairs lieutenant for the Miami-Dade Police Department, working in the division that investigates allegations of wrongdoing by cops. Outside the office, authorities allege that the 45-year-old longtime officer worked with a drug trafficking organization to help plan a murder plot and get guns. A criminal complaint unsealed in U.S. District Court in New Jersey Tuesday accuses Mata, also known as ""The Milk Man,"" of using his role as a police officer to he...","b'Criminal complaint: Cop used his role to help cocaine traffickers.\nRalph Mata, an internal affairs lieutenant, allegedly helped group get guns.\nHe also arranged to pay two assassins in a murder plot, a complaint alleges.'"
2,"b""A drunk driver who killed a young woman in a head-on crash while checking his mobile phone has been jailed for six years. Craig Eccleston-Todd, 27, was driving home from a night at a pub when he received a text message. As he was reading or replying to it, he veered across the road while driving round a bend and smashed into Rachel Titley\xe2\x80\x99s car coming the other way. Craig Eccleston-Todd, 27 (left) was using his mobile phone when he crashed head-on into the car being driven by Ra...","b""Craig Eccleston-Todd, 27, had drunk at least three pints before driving car.\nWas using phone when he veered across road in Yarmouth, Isle of Wight.\nCrashed head-on into 28-year-old Rachel Titley's car, who died in hospital.\nPolice say he would have been over legal drink-drive limit at time of crash.\nHe was found guilty at Portsmouth Crown Court of causing death by dangerous driving."""


(D) Char Words Converstion

In [ ]:
CHAT_WORDS_STR = """
AFAIK=As Far As I Know
AFK=Away From Keyboard
ASAP=As Soon As Possible
ATK=At The Keyboard
ATM=At The Moment
A3=Anytime, Anywhere, Anyplace
BAK=Back At Keyboard
BBL=Be Back Later
BBS=Be Back Soon
BFN=Bye For Now
B4N=Bye For Now
BRB=Be Right Back
BRT=Be Right There
BTW=By The Way
B4=Before
B4N=Bye For Now
CU=See You
CUL8R=See You Later
CYA=See You
FAQ=Frequently Asked Questions
FC=Fingers Crossed
FWIW=For What It's Worth
FYI=For Your Information
GAL=Get A Life
GG=Good Game
GN=Good Night
GMTA=Great Minds Think Alike
GR8=Great!
G9=Genius
IC=I See
ICQ=I Seek you (also a chat program)
ILU=ILU: I Love You
IMHO=In My Honest/Humble Opinion
IMO=In My Opinion
IOW=In Other Words
IRL=In Real Life
KISS=Keep It Simple, Stupid
LDR=Long Distance Relationship
LMAO=Laugh My A.. Off
LOL=Laughing Out Loud
LTNS=Long Time No See
L8R=Later
MTE=My Thoughts Exactly
M8=Mate
NRN=No Reply Necessary
OIC=Oh I See
PITA=Pain In The A..
PRT=Party
PRW=Parents Are Watching
ROFL=Rolling On The Floor Laughing
ROFLOL=Rolling On The Floor Laughing Out Loud
ROTFLMAO=Rolling On The Floor Laughing My A.. Off
SK8=Skate
STATS=Your sex and age
ASL=Age, Sex, Location
THX=Thank You
TTFN=Ta-Ta For Now!
TTYL=Talk To You Later
U=You
U2=You Too
U4E=Yours For Ever
WB=Welcome Back
WTF=What The F...
WTG=Way To Go!
WUF=Where Are You From?
W8=Wait...
7K=Sick:-D Laugher
"""

In [ ]:
# First, we're going to convert this long string into set of words and its shortcut
chat_words_map_dict = {}
chat_shortcut_list = set()
for line in CHAT_WORDS_STR.split("\n"):
    if line != '':
        shortcut = line.split('=')[0] # split the line from `=` sign and select shortcut
        chat_words = line.split('=')[1]
        chat_shortcut_list.add(shortcut) # add the chat  shortcut to the set
        chat_words_map_dict[shortcut] = chat_words # add each chat_words corresponding to its shortcut

chat_words_map_dict

{'AFAIK': 'As Far As I Know',
 'AFK': 'Away From Keyboard',
 'ASAP': 'As Soon As Possible',
 'ATK': 'At The Keyboard',
 'ATM': 'At The Moment',
 'A3': 'Anytime, Anywhere, Anyplace',
 'BAK': 'Back At Keyboard',
 'BBL': 'Be Back Later',
 'BBS': 'Be Back Soon',
 'BFN': 'Bye For Now',
 'B4N': 'Bye For Now',
 'BRB': 'Be Right Back',
 'BRT': 'Be Right There',
 'BTW': 'By The Way',
 'B4': 'Before',
 'CU': 'See You',
 'CUL8R': 'See You Later',
 'CYA': 'See You',
 'FAQ': 'Frequently Asked Questions',
 'FC': 'Fingers Crossed',
 'FWIW': "For What It's Worth",
 'FYI': 'For Your Information',
 'GAL': 'Get A Life',
 'GG': 'Good Game',
 'GN': 'Good Night',
 'GMTA': 'Great Minds Think Alike',
 'GR8': 'Great!',
 'G9': 'Genius',
 'IC': 'I See',
 'ICQ': 'I Seek you (also a chat program)',
 'ILU': 'ILU: I Love You',
 'IMHO': 'In My Honest/Humble Opinion',
 'IMO': 'In My Opinion',
 'IOW': 'In Other Words',
 'IRL': 'In Real Life',
 'KISS': 'Keep It Simple, Stupid',
 'LDR': 'Long Distance Relationship',
 'LM

In [ ]:
def chat_words_conversion(text: Text) -> Text:
    new_text = []
    for word in text.split():
        if word.upper() in chat_words_map_dict:
            new_text.append(chat_words_map_dict[word.upper()])
        else:
            new_text.append(word)
    return " ".join(new_text)

In [ ]:
df['text'] = df['text'].parallel_apply(lambda sentence: chat_words_conversion(sentence))
df['summary'] = df['summary'].parallel_apply(lambda sentence: chat_words_conversion(sentence))

In [ ]:
df.head(3)

,text,summary
0,"b""By. Associated Press. PUBLISHED:. 14:11 EST, 25 October 2013. |. UPDATED:. 15:36 EST, 25 October 2013. The bishop of the Fargo Catholic Diocese in North Dakota has exposed potentially hundreds of church members in Fargo, Grand Forks and Jamestown to the hepatitis A virus in late September and early October. The state Health Department has issued an advisory of exposure for anyone who attended five churches and took communion. Bishop John Folda (pictured) of the Fargo Catholic Diocese in No...","b'Bishop John Folda, of North Dakota, is taking time off after being diagnosed.\nHe contracted the infection through contaminated food in Italy.\nChurch members in Fargo, Grand Forks and Jamestown could have been exposed.'"
1,"b'(CNN) -- Ralph Mata was an internal affairs lieutenant for the Miami-Dade Police Department, working in the division that investigates allegations of wrongdoing by cops. Outside the office, authorities allege that the 45-year-old longtime officer worked with a drug trafficking organization to help plan a murder plot and get guns. A criminal complaint unsealed in U.S. District Court in New Jersey Tuesday accuses Mata, also known as ""The Milk Man,"" of using his role as a police officer to he...","b'Criminal complaint: Cop used his role to help cocaine traffickers.\nRalph Mata, an internal affairs lieutenant, allegedly helped group get guns.\nHe also arranged to pay two assassins in a murder plot, a complaint alleges.'"
2,"b""A drunk driver who killed a young woman in a head-on crash while checking his mobile phone has been jailed for six years. Craig Eccleston-Todd, 27, was driving home from a night at a pub when he received a text message. As he was reading or replying to it, he veered across the road while driving round a bend and smashed into Rachel Titley\xe2\x80\x99s car coming the other way. Craig Eccleston-Todd, 27 (left) was using his mobile phone when he crashed head-on into the car being driven by Ra...","b""Craig Eccleston-Todd, 27, had drunk at least three pints before driving car.\nWas using phone when he veered across road in Yarmouth, Isle of Wight.\nCrashed head-on into 28-year-old Rachel Titley's car, who died in hospital.\nPolice say he would have been over legal drink-drive limit at time of crash.\nHe was found guilty at Portsmouth Crown Court of causing death by dangerous driving."""


(E) English contractions

What is a contraction?

A contraction is a shortened form of a word (or group of words) that omits certain letters or sounds. In most contractions, an apostrophe represents the missing letters. The most common contractions are made up of verbs, auxiliaries, or modals attached to other words: He would=He’d. I have=I’ve. They are=They’re. You cannot=You can’t.

In [ ]:
def en_contractions(text: Text) -> Text:
    return ' '.join([contractions.fix(word)
                     if word in contractions.contractions_dict else word
                     for word in text.split()])

In [ ]:
df['text'] = df['text'].parallel_apply(lambda sentence: en_contractions(sentence))
df['summary'] = df['summary'].parallel_apply(lambda sentence: en_contractions(sentence))

In [ ]:
df.head(3)

,text,summary
0,"b""By. Associated Press. PUBLISHED:. 14:11 EST, 25 October 2013. |. UPDATED:. 15:36 EST, 25 October 2013. The bishop of the Fargo Catholic Diocese in North Dakota has exposed potentially hundreds of church members in Fargo, Grand Forks and Jamestown to the hepatitis A virus in late September and early October. The state Health Department has issued an advisory of exposure for anyone who attended five churches and took communion. Bishop John Folda (pictured) of the Fargo Catholic Diocese in No...","b'Bishop John Folda, of North Dakota, is taking time off after being diagnosed.\nHe contracted the infection through contaminated food in Italy.\nChurch members in Fargo, Grand Forks and Jamestown could have been exposed.'"
1,"b'(CNN) -- Ralph Mata was an internal affairs lieutenant for the Miami-Dade Police Department, working in the division that investigates allegations of wrongdoing by cops. Outside the office, authorities allege that the 45-year-old longtime officer worked with a drug trafficking organization to help plan a murder plot and get guns. A criminal complaint unsealed in U.S. District Court in New Jersey Tuesday accuses Mata, also known as ""The Milk Man,"" of using his role as a police officer to he...","b'Criminal complaint: Cop used his role to help cocaine traffickers.\nRalph Mata, an internal affairs lieutenant, allegedly helped group get guns.\nHe also arranged to pay two assassins in a murder plot, a complaint alleges.'"
2,"b""A drunk driver who killed a young woman in a head-on crash while checking his mobile phone has been jailed for six years. Craig Eccleston-Todd, 27, was driving home from a night at a pub when he received a text message. As he was reading or replying to it, he veered across the road while driving round a bend and smashed into Rachel Titley\xe2\x80\x99s car coming the other way. Craig Eccleston-Todd, 27 (left) was using his mobile phone when he crashed head-on into the car being driven by Ra...","b""Craig Eccleston-Todd, 27, had drunk at least three pints before driving car.\nWas using phone when he veered across road in Yarmouth, Isle of Wight.\nCrashed head-on into 28-year-old Rachel Titley's car, who died in hospital.\nPolice say he would have been over legal drink-drive limit at time of crash.\nHe was found guilty at Portsmouth Crown Court of causing death by dangerous driving."""


(F) Dealing with Special Characters

In [ ]:
s_chars = '¥₽ÏïŰŬĎŸæ₿₪ÚŇÀèÅ”ĜåŽÖéříÿý€ŝĤ₹áŜŮÂ₴ûÌÇšŘúüëÓ₫ŠčÎŤÆÒœ₩öËäøÍťìĈôàĥÝ¢ç“žðÙÊĉŭÈŒÐÉÔĵùÁů„âÄűĴóêĝÞîØòď฿ČÜþňÛ'
PUNC = '+@«#_\-!$%%^&*¬()£<>?/\\|}\]\[{;\,~:\"\''

In [ ]:
def special_char(text: Text) -> Text:
    # first, let's remove any unicode strings
    text = text.encode('ascii', 'ignore').decode()
    # remove printable bachslashes
    text = re.sub(r'[\t\s\n\r\b\a]', ' ', text)
    # Special letters
    text = re.sub(r'[{}]'.format(s_chars), '', text)
    # Punctuation [remove punctuation between spaces only which represent noises]
    text = re.sub(r'\s[{}]\s'.format(PUNC), ' ', text)
    # space at the start or the end of the context
    text = re.sub(r'(^\s)|(\s$)', '', text)
    # Single character
    text = re.sub(r'(\s[^iIaA]\s)', ' ', text)
    return text

In [ ]:
df['text'] = df['text'].parallel_apply(lambda sentence: special_char(sentence))
df['summary'] = df['summary'].parallel_apply(lambda sentence: special_char(sentence))

In [ ]:
df.head(3)

,text,summary
0,"b""By. Associated Press. PUBLISHED:. 14:11 EST, 25 October 2013. |. UPDATED:. 15:36 EST, 25 October 2013. The bishop of the Fargo Catholic Diocese in North Dakota has exposed potentially hundreds of church members in Fargo, Grand Forks and Jamestown to the hepatitis A virus in late September and early October. The state Health Department has issued an advisory of exposure for anyone who attended five churches and took communion. Bishop John Folda (pictured) of the Fargo Catholic Diocese in No...","b'Bishop John Folda, of North Dakota, is taking time off after being diagnosed.\nHe contracted the infection through contaminated food in Italy.\nChurch members in Fargo, Grand Forks and Jamestown could have been exposed.'"
1,"b'(CNN) -- Ralph Mata was an internal affairs lieutenant for the Miami-Dade Police Department, working in the division that investigates allegations of wrongdoing by cops. Outside the office, authorities allege that the 45-year-old longtime officer worked with a drug trafficking organization to help plan a murder plot and get guns. A criminal complaint unsealed in U.S. District Court in New Jersey Tuesday accuses Mata, also known as ""The Milk Man,"" of using his role as a police officer to he...","b'Criminal complaint: Cop used his role to help cocaine traffickers.\nRalph Mata, an internal affairs lieutenant, allegedly helped group get guns.\nHe also arranged to pay two assassins in a murder plot, a complaint alleges.'"
2,"b""A drunk driver who killed a young woman in a head-on crash while checking his mobile phone has been jailed for six years. Craig Eccleston-Todd, 27, was driving home from a night at a pub when he received a text message. As he was reading or replying to it, he veered across the road while driving round a bend and smashed into Rachel Titley\xe2\x80\x99s car coming the other way. Craig Eccleston-Todd, 27 (left) was using his mobile phone when he crashed head-on into the car being driven by Ra...","b""Craig Eccleston-Todd, 27, had drunk at least three pints before driving car.\nWas using phone when he veered across road in Yarmouth, Isle of Wight.\nCrashed head-on into 28-year-old Rachel Titley's car, who died in hospital.\nPolice say he would have been over legal drink-drive limit at time of crash.\nHe was found guilty at Portsmouth Crown Court of causing death by dangerous driving."""


(G) Dealing with Special cases

In [ ]:
df.head()

,text,summary
0,"b""By. Associated Press. PUBLISHED:. 14:11 EST, 25 October 2013. |. UPDATED:. 15:36 EST, 25 October 2013. The bishop of the Fargo Catholic Diocese in North Dakota has exposed potentially hundreds of church members in Fargo, Grand Forks and Jamestown to the hepatitis A virus in late September and early October. The state Health Department has issued an advisory of exposure for anyone who attended five churches and took communion. Bishop John Folda (pictured) of the Fargo Catholic Diocese in No...","b'Bishop John Folda, of North Dakota, is taking time off after being diagnosed.\nHe contracted the infection through contaminated food in Italy.\nChurch members in Fargo, Grand Forks and Jamestown could have been exposed.'"
1,"b'(CNN) -- Ralph Mata was an internal affairs lieutenant for the Miami-Dade Police Department, working in the division that investigates allegations of wrongdoing by cops. Outside the office, authorities allege that the 45-year-old longtime officer worked with a drug trafficking organization to help plan a murder plot and get guns. A criminal complaint unsealed in U.S. District Court in New Jersey Tuesday accuses Mata, also known as ""The Milk Man,"" of using his role as a police officer to he...","b'Criminal complaint: Cop used his role to help cocaine traffickers.\nRalph Mata, an internal affairs lieutenant, allegedly helped group get guns.\nHe also arranged to pay two assassins in a murder plot, a complaint alleges.'"
2,"b""A drunk driver who killed a young woman in a head-on crash while checking his mobile phone has been jailed for six years. Craig Eccleston-Todd, 27, was driving home from a night at a pub when he received a text message. As he was reading or replying to it, he veered across the road while driving round a bend and smashed into Rachel Titley\xe2\x80\x99s car coming the other way. Craig Eccleston-Todd, 27 (left) was using his mobile phone when he crashed head-on into the car being driven by Ra...","b""Craig Eccleston-Todd, 27, had drunk at least three pints before driving car.\nWas using phone when he veered across road in Yarmouth, Isle of Wight.\nCrashed head-on into 28-year-old Rachel Titley's car, who died in hospital.\nPolice say he would have been over legal drink-drive limit at time of crash.\nHe was found guilty at Portsmouth Crown Court of causing death by dangerous driving."""
3,"b""(CNN) -- With a breezy sweep of his pen President Vladimir Putin wrote a new chapter into Crimea's turbulent history, committing the region to a future returned to Russian domain. Sixty years prior, Ukraine's breakaway peninsula was signed away just as swiftly by Soviet leader Nikita Khrushchev. But dealing with such a blatant land grab on its eastern flank will not be anywhere near as quick and easy for Europe's 28-member union. Because, unlike Crimea's rushed referendum, everyone has a s...","b""Nina dos Santos says Europe must be ready to accept sanctions will hurt both sides.\nTargeting Russia's business community would be one way of sapping their support for President Putin, she says.\nBut she says Europe would have a hard time keeping its factories going without power from the east."""
4,"b""Fleetwood are the only team still to have a 100% record in Sky Bet League One as a 2-0 win over Scunthorpe sent Graham Alexander\xe2\x80\x99s men top of the table. The Cod Army are playing in the third tier for the first time in their history after six promotions in nine years and their remarkable ascent shows no sign of slowing with Jamie Proctor and Gareth Evans scoring the goals at Glanford Park. Fleetwood were one of five teams to have won two out of two but the other four clubs Peterb...","b'Fleetwood top of League One after 2-0 win at Scunthorpe.\nPeterborough, Bristol City, Chesterfield and Crawley all drop first points of the season.\nStand-in striker Matt Done scores a hat-trick as Rochdale thrash Crewe 5-2.\nWins for Notts County and Yeovil.\nCoventry/Bradford and Oldham/Port Vale bot

In [ ]:
def datasetGeneralReplace(old, new, regexValue):
  df['text'] = df['text'].str.replace(old, new, regex=regexValue)
  df['summary'] = df['summary'].str.replace(old, new, regex=regexValue)


In [ ]:
datasetGeneralReplace("yr", 'year',False)
datasetGeneralReplace("...", ' ',False)
datasetGeneralReplace('-->', '',False)
datasetGeneralReplace("b'", '',False)
datasetGeneralReplace('b"', '',False)
datasetGeneralReplace('-', '',False)
datasetGeneralReplace('--', '',False)
datasetGeneralReplace('=', '',False)
datasetGeneralReplace('==', '',False)
datasetGeneralReplace('#', '',False)
datasetGeneralReplace('##', '',False)
datasetGeneralReplace('#-#', '',False)
datasetGeneralReplace('."', '.',False)
datasetGeneralReplace(".'", '.',False)
datasetGeneralReplace(" '", "'",False)

datasetGeneralReplace(r"(\s[^iIaA]\s)", ' ', True)
datasetGeneralReplace(r'\s(\w$)|\s(\w[^me]$)', '', True)
datasetGeneralReplace(r'(^\s)|(\s$)', '', True)
datasetGeneralReplace(r'(^\s)|(\s$)', '', True)
datasetGeneralReplace(r"(\si\s)", ' I ', True)
# datasetGeneralReplace()
# datasetGeneralReplace()
# datasetGeneralReplace()


In [ ]:
df.head(2)

,text,summary
0,"By. Associated Press. PUBLISHED:. 14:11 EST, 25 October 2013. |. UPDATED:. 15:36 EST, 25 October 2013. The bishop of the Fargo Catholic Diocese in North Dakota has exposed potentially hundreds of church members in Fargo, Grand Forks and Jamestown to the hepatitis A virus in late September and early October. The state Health Department has issued an advisory of exposure for anyone who attended five churches and took communion. Bishop John Folda (pictured) of the Fargo Catholic Diocese in Nort...","Bishop John Folda, of North Dakota, is taking time off after being diagnosed.\nHe contracted the infection through contaminated food in Italy.\nChurch members in Fargo, Grand Forks and Jamestown could have been exposed."
1,"(CNN) Ralph Mata was an internal affairs lieutenant for the MiamiDade Police Department, working in the division that investigates allegations of wrongdoing by cops. Outside the office, authorities allege that the 45yearold longtime officer worked with a drug trafficking organization to help plan a murder plot and get guns. A criminal complaint unsealed in U.S. District Court in New Jersey Tuesday accuses Mata, also known as ""The Milk Man,"" of using his role as a police officer to help the ...","Criminal complaint: Cop used his role to help cocaine traffickers.\nRalph Mata, an internal affairs lieutenant, allegedly helped group get guns.\nHe also arranged to pay two assassins in a murder plot, a complaint alleges."


(H) Spell Correction

In [ ]:
!pip install textblob

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from textblob import TextBlob

def spell_correction(df):
    # creating a new column for the corrected text
    df['corrected_text'] = df['text']
    # creating a new column for the corrected summary
    df['corrected_summary'] = df['summary']
    # creating a for loop for the entire dataset
    for i in range(len(df)):
        # Records
        print('Counter: ' + str(i+1) + '/' + str(len(df)+1))
        # creating a variable for the text of the current row
        text = df['corrected_text'][i]
        # creating a variable for the summary of the current row
        summary = df['corrected_summary'][i]
        # creating a variable for the corrected text of the current row
        corrected_text = TextBlob(text).correct()
        # creating a variable for the corrected summary of the current row
        corrected_summary = TextBlob(summary).correct()
        # updating the corrected text column with the corrected text
        df['corrected_text'][i] = str(corrected_text)
        # updating the corrected summary column with the corrected summary
        df['corrected_summary'][i] = str(corrected_summary)
    # returning the dataset with the new columns
    return df

spell_correction(df)
df_copy_correction = df.copy()

Counter: 1/51
Counter: 2/51
Counter: 3/51
Counter: 4/51
Counter: 5/51
Counter: 6/51
Counter: 7/51
Counter: 8/51
Counter: 9/51
Counter: 10/51
Counter: 11/51
Counter: 12/51
Counter: 13/51
Counter: 14/51
Counter: 15/51
Counter: 16/51
Counter: 17/51
Counter: 18/51
Counter: 19/51
Counter: 20/51
Counter: 21/51
Counter: 22/51
Counter: 23/51
Counter: 24/51
Counter: 25/51
Counter: 26/51
Counter: 27/51
Counter: 28/51
Counter: 29/51
Counter: 30/51
Counter: 31/51
Counter: 32/51
Counter: 33/51
Counter: 34/51
Counter: 35/51
Counter: 36/51
Counter: 37/51
Counter: 38/51
Counter: 39/51
Counter: 40/51
Counter: 41/51
Counter: 42/51
Counter: 43/51
Counter: 44/51
Counter: 45/51
Counter: 46/51
Counter: 47/51
Counter: 48/51
Counter: 49/51
Counter: 50/51


In [ ]:
df_copy_correction.head(2)

,text,summary,corrected_text,corrected_summary
0,"By. Associated Press. PUBLISHED:. 14:11 EST, 25 October 2013. |. UPDATED:. 15:36 EST, 25 October 2013. The bishop of the Fargo Catholic Diocese in North Dakota has exposed potentially hundreds of church members in Fargo, Grand Forks and Jamestown to the hepatitis A virus in late September and early October. The state Health Department has issued an advisory of exposure for anyone who attended five churches and took communion. Bishop John Folda (pictured) of the Fargo Catholic Diocese in Nort...","Bishop John Folda, of North Dakota, is taking time off after being diagnosed.\nHe contracted the infection through contaminated food in Italy.\nChurch members in Fargo, Grand Forks and Jamestown could have been exposed.","By. Associated Dress. PUBLISHED:. 14:1 EST, 25 October 2013. |. UPDATED:. 15:36 EST, 25 October 2013. The bishop of the Cargo Catholic Diocese in North Dakota has exposed potentially hundreds of church members in Cargo, Grand Works and Jamestown to the keratitis A virus in late September and early October. The state Health Department has issued an advisory of exposure for anyone who attended five churches and took communion. Bishop John Old (pictured) of the Cargo Catholic Diocese in North D...","Bishop John Old, of North Dakota, is taking time off after being diagnosed.\ne contracted the infection through contaminated food in Italy.\church members in Cargo, Grand Works and Jamestown could have been exposed."
1,"(CNN) Ralph Mata was an internal affairs lieutenant for the MiamiDade Police Department, working in the division that investigates allegations of wrongdoing by cops. Outside the office, authorities allege that the 45yearold longtime officer worked with a drug trafficking organization to help plan a murder plot and get guns. A criminal complaint unsealed in U.S. District Court in New Jersey Tuesday accuses Mata, also known as ""The Milk Man,"" of using his role as a police officer to help the ...","Criminal complaint: Cop used his role to help cocaine traffickers.\nRalph Mata, an internal affairs lieutenant, allegedly helped group get guns.\nHe also arranged to pay two assassins in a murder plot, a complaint alleges.","(CNN) Ralph Data was an internal affairs lieutenant for the MiamiDade Police Department, working in the division that investigate allegation of wrongdoing by corps. Outside the office, authorities allege that the 45yearold longtime officer worked with a drug trafficking organization to help plan a murder plot and get guns. A criminal complaint sealed in U.S. District Court in New Jersey Tuesday accuses Data, also known as ""The Silk An,"" of using his role as a police officer to help the drug...","Criminal complaint: Top used his role to help cocaine traffickers.\ralph Data, an internal affairs lieutenant, allegedly helped group get guns.\ne also arranged to pay two assins in a murder plot, a complaint alleged."


In [ ]:
df_copy_correction_copy = df_copy_correction.copy()

Reduce consecutive words in sentence

In [ ]:
def rm_duplicates(text: Text) -> Text:
    return re.sub(r'\b(\w+\s*)\1{1,}', '\\1', text)

In [ ]:
df_copy_correction['corrected_text'] = df_copy_correction['corrected_text'].parallel_apply(lambda sentence: rm_duplicates(sentence))
df_copy_correction['corrected_summary'] = df_copy_correction['corrected_summary'].parallel_apply(lambda sentence: rm_duplicates(sentence))

In [ ]:
df_copy_correction.tail()

,text,summary,corrected_text,corrected_summary
45,"Luke Anderson, winner of Big Brother 2012, was born a woman but is undergoing a full sex change. Wife Becki says they have an'incredible sex life' and cannot wait to start a family. His hormone injections have lowered his. voice, grown hair on his body and even thickened his facial bones, which. help him achieve a more masculine look. By. Bianca London. PUBLISHED:. 07:21 EST, 28 December 2012. |. UPDATED:. 07:32 EST, 29 December 2012. Luke Anderson won over the nation when he was crowned the...","Luke Anderson, winner of Big Brother 2012, was born a woman but is undergoing a full sex change.\nWife Becki says they have an'incredible sex life' and cannot wait to start a family.\nHis hormone injections have lowered his.\nvoice, grown hair on his body and even thickened his facial bones, which.\nhelp him achieve a more masculine look.","Duke Anderson, winner of Fig Brother 2012, was born a woman but is undergoing a full sex change. Life Neck says they have an'incredible sex life' and cannot wait to start a family. His harmony injections have lowered his. voice, grown hair on his body and even thickened his facial bones, which. help him achieve a more masculine look. By. Diana London. PUBLISHED:. 07:21 EST, 28 December 2012. |. UPDATED:. 07:32 EST, 29 December 2012. Duke Anderson won over the nation when he was crowned the w...","Duke Anderson, winner of Fig Brother 2012, was born a woman but is undergoing a full sex change.\life Neck says they have an'incredible sex life' and cannot wait to start a family.\his harmony injections have lowered his.\voice, grown hair on his body and even thickened his facial bones, which.\help him achieve a more masculine look."
46,"A furniture shop refused to repair a pensioner\'s armchairs after claiming she had ruined them with sweat caused by her medication. Grandmother Barbara Challons, of Mosborough, South Yorkshire, bought the two twoseater armchairs from Harveys Furniture Store three years ago and says they have always had problems. The company have sent her several spare parts but after repeated complaints they finally called at her home to fix them. A furniture shop refused to repair armchairs owned by Barbara...",Barbara Challons bought two armchairs from Harveys Furniture Store three years ago.\nPensioner claims chairs have both been plagued by problems ever since.\nShe confirmed to an inspector from the store that she was on medication.\nBut she then received letter saying medication had caused her to sweat and'contaminated' the \xc2\xa3500 chairs.\nThe letter explained to 79yearold that this meant staff could not repair them.\nFormer auxiliary nurse describes comments as'personal attack on me and ...,"A furniture shop refused to repair a pensioner\'s armchairs after claiming she had ruined them with sweat caused by her meditation. Grandmother Barbara Allons, of Mosborough, South Yorkshire, bought the two twoseater armchairs from Parleys Furniture Tore three years ago and says they have always had problems. The company have sent her several spare parts but after repeated complaints they finally called at her home to fix them. A furniture shop refused to repair armchairs owned by Barbara Al...","Barbara Allons bought two armchairs from Parleys Furniture Tore three years ago.\pensioner claims chairs have both been plagued by problems ever since.\the confirmed to an inspector from the store that she was on meditation.\nut she then received letter saying meditation had caused her to sweat and'contaminated' the \act\xa3500 chairs.\the letter explained to 79yearold that this meant staff could not repair them.\former axillary nurse describes comments as'personal attack on me and my health'"""
47,"Sony says it will release the Seth RogenJames Franco comedy'The Interview' mere days after it announced the film would not in theaters for its scheduled Christmas release date. Sony cancelled the film's release last week after the hackers threatened re

Check the number of misspelt words before && after Applying the spell correction

In [ ]:
spelling = SpellChecker(distance=1)
def misspelt_words_fn(dataframe: pd.DataFrame, column: Text) -> Set[Text]:
    misspelt_words = set()
    for text in dataframe[column].str.casefold():
        [misspelt_words.add(word) for word in spelling.unknown(text.split())]
    return misspelt_words

In [ ]:
misspelt_text_before = misspelt_words_fn(df_copy_correction, 'text')
misspelt_summary_before = misspelt_words_fn(df_copy_correction, 'summary')
print(f'Text misspelt: {len(misspelt_text_before)}\nSummary misspelt: {len(misspelt_summary_before)}')

Text misspelt: 3858
Summary misspelt: 417


In [ ]:
misspelt_text_after = misspelt_words_fn(df_copy_correction, 'corrected_text')
misspelt_summary_after = misspelt_words_fn(df_copy_correction, 'corrected_summary')
print(f'Text misspelt: {len(misspelt_text_after)}\nSummary misspelt: {len(misspelt_summary_after)}')

Text misspelt: 3628
Summary misspelt: 383


Adjustments after the Correction process

In [ ]:
def datasetGeneralReplaceCorrected(old, new, regexValue):
  df_copy_correction['corrected_text'] = df_copy_correction['corrected_text'].str.replace(old, new, regex=regexValue)
  df_copy_correction['corrected_summary'] = df_copy_correction['corrected_summary'].str.replace(old, new, regex=regexValue)


In [ ]:
datasetGeneralReplaceCorrected(r"(\si\s)", ' I ', True)
datasetGeneralReplaceCorrected(r"(q\s)", '', True)
datasetGeneralReplaceCorrected(r"(\s[^iIaA]\s)", ' ', True)
datasetGeneralReplaceCorrected(r"(\s[^iIaA]\s)", ' ', True)

In [ ]:
df_copy_correction.head()

,text,summary,corrected_text,corrected_summary
0,"By. Associated Press. PUBLISHED:. 14:11 EST, 25 October 2013. |. UPDATED:. 15:36 EST, 25 October 2013. The bishop of the Fargo Catholic Diocese in North Dakota has exposed potentially hundreds of church members in Fargo, Grand Forks and Jamestown to the hepatitis A virus in late September and early October. The state Health Department has issued an advisory of exposure for anyone who attended five churches and took communion. Bishop John Folda (pictured) of the Fargo Catholic Diocese in Nort...","Bishop John Folda, of North Dakota, is taking time off after being diagnosed.\nHe contracted the infection through contaminated food in Italy.\nChurch members in Fargo, Grand Forks and Jamestown could have been exposed.","By. Associated Dress. PUBLISHED:. 14:1 EST, 25 October 2013. |. UPDATED:. 15:36 EST, 25 October 2013. The bishop of the Cargo Catholic Diocese in North Dakota has exposed potentially hundreds of church members in Cargo, Grand Works and Jamestown to the keratitis A virus in late September and early October. The state Health Department has issued an advisory of exposure for anyone who attended five churches and took communion. Bishop John Old (pictured) of the Cargo Catholic Diocese in North D...","Bishop John Old, of North Dakota, is taking time off after being diagnosed.\ne contracted the infection through contaminated food in Italy.\church members in Cargo, Grand Works and Jamestown could have been exposed."
1,"(CNN) Ralph Mata was an internal affairs lieutenant for the MiamiDade Police Department, working in the division that investigates allegations of wrongdoing by cops. Outside the office, authorities allege that the 45yearold longtime officer worked with a drug trafficking organization to help plan a murder plot and get guns. A criminal complaint unsealed in U.S. District Court in New Jersey Tuesday accuses Mata, also known as ""The Milk Man,"" of using his role as a police officer to help the ...","Criminal complaint: Cop used his role to help cocaine traffickers.\nRalph Mata, an internal affairs lieutenant, allegedly helped group get guns.\nHe also arranged to pay two assassins in a murder plot, a complaint alleges.","(CNN) Ralph Data was an internal affairs lieutenant for the MiamiDade Police Department, working in the division that investigate allegation of wrongdoing by corps. Outside the office, authorities allege that the 45yearold longtime officer worked with a drug trafficking organization to help plan a murder plot and get guns. A criminal complaint sealed in U.S. District Court in New Jersey Tuesday accuses Data, also known as ""The Silk An,"" of using his role as a police officer to help the drug...","Criminal complaint: Top used his role to help cocaine traffickers.\ralph Data, an internal affairs lieutenant, allegedly helped group get guns.\ne also arranged to pay two assins in a murder plot, a complaint alleged."
2,"A drunk driver who killed a young woman in a headon crash while checking his mobile phone has been jailed for six years. Craig EcclestonTodd, 27, was driving home from a night at a pub when he received a text message. As he was reading or replying to it, he veered across the road while driving round a bend and smashed into Rachel Titley\xe2\x80\x99s car coming the other way. Craig EcclestonTodd, 27 (left) was using his mobile phone when he crashed headon into the car being driven by Rachel T...","Craig EcclestonTodd, 27, had drunk at least three pints before driving car.\nWas using phone when he veered across road in Yarmouth, Isle of Wight.\nCrashed headon into 28yearold Rachel Titley's car, who died in hospital.\nPolice say he would have been over legal drinkdrive limit at time of crash.\nHe was found guilty at Portsmouth Crown Court of causing death by dangerous driving.","A drunk driver who killed a young woman in a head crash while checking his mobile phone has been jailed for six years. Craig EcclestonTodd, 27, was driving home f

(I) Restore Punctuation

Punctuations are important for having semantic context when you are applying NLP – especially if you are creating generative models. This will allow you to create a model that has the same capability of writing as human writing. Therefore, we need a punctuator model which allows us to restore the punctuation in our sentences. Hence, we're going to use a model called Punctuator.

In [ ]:
args = InferenceArguments(
              model_name_or_path="Qishuai/distilbert_punctuator_en",
              tokenizer_name="Qishuai/distilbert_punctuator_en",
              tag2punctuator=DEFAULT_ENGLISH_TAG_PUNCTUATOR_MAP,
)
inference = Inference(inference_args=args, verbose=False)

In [ ]:
def punct_restoration(list_of_text: List[Text], name: Text) -> List[Text]:
    list_of_texts = []
    for text in tqdm(list_of_text, desc=f"Auto Punctuation for {name}"):
        list_of_texts.append(
            inference.punctuation([text])[0][0]
        )
    return list_of_texts

In [ ]:
df_copy_correction['punc_corrected_text'] = punct_restoration(df_copy_correction['corrected_text'].values.tolist(), "text")
df_copy_correction['punc_corrected_summary'] = punct_restoration(df_copy_correction['corrected_summary'].values.tolist(), 'summary')

Auto Punctuation for text:   0%|          | 0/50 [00:00<?, ?it/s]

Auto Punctuation for summary:   0%|          | 0/50 [00:00<?, ?it/s]

(J) Grammerly Correction

We're going to use a library called Gramformer

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.huggingface/token
Login successful


In [ ]:
# Controlling random samples
def set_seed(seed):
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

In [ ]:
set_seed(1212)
gf = Gramformer(models = 1, use_gpu=False) # 1=corrector, 2=detector

[Gramformer] Grammar error correct/highlight model loaded..


In [ ]:
!pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Ref: https://stackoverflow.com/a/4576110/14473118
import nltk.data
nltk.download('punkt')
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
def grammely_correction(list_text: List[Text], name: Text) -> List[Text]:
    list_of_correction = []
    for text in tqdm(list_text, desc=f'Grammerly Correction for {name}'):
        if len(text.split()) < 50:
            list_of_correction.append(list(gf.correct(text, max_candidates=1))[0])
        else:
            list_of_correction.append(" ".join([list(gf.correct(sentence, max_candidates=1))[0]
                                                for sentence in tokenizer.tokenize(text)]))
    return list_of_correction

In [ ]:
df_copy_correction['gram_corrected_text'] = grammely_correction(df_copy_correction['punc_corrected_text'].values.tolist(), "text")
df_copy_correction['gram_corrected_summary'] = grammely_correction(df_copy_correction['punc_corrected_summary'].values.tolist(), 'summary')

Grammerly Correction for text:   0%|          | 0/50 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

In [ ]:
df_copy_correction

,text,summary,corrected_text,corrected_summary,punc_corrected_text,punc_corrected_summary,gram_corrected_text,gram_corrected_summary
0,"By. Associated Press. PUBLISHED:. 14:11 EST, 25 October 2013. |. UPDATED:. 15:36 EST, 25 October 2013. The bishop of the Fargo Catholic Diocese in North Dakota has exposed potentially hundreds of church members in Fargo, Grand Forks and Jamestown to the hepatitis A virus in late September and early October. The state Health Department has issued an advisory of exposure for anyone who attended five churches and took communion. Bishop John Folda (pictured) of the Fargo Catholic Diocese in Nort...","Bishop John Folda, of North Dakota, is taking time off after being diagnosed.\nHe contracted the infection through contaminated food in Italy.\nChurch members in Fargo, Grand Forks and Jamestown could have been exposed.","By. Associated Dress. PUBLISHED:. 14:1 EST, 25 October 2013. |. UPDATED:. 15:36 EST, 25 October 2013. The bishop of the Cargo Catholic Diocese in North Dakota has exposed potentially hundreds of church members in Cargo, Grand Works and Jamestown to the keratitis A virus in late September and early October. The state Health Department has issued an advisory of exposure for anyone who attended five churches and took communion. Bishop John Old (pictured) of the Cargo Catholic Diocese in North D...","Bishop John Old, of North Dakota, is taking time off after being diagnosed.\ne contracted the infection through contaminated food in Italy.\church members in Cargo, Grand Works and Jamestown could have been exposed.","By. Associated Dress. PUBLISHED:. 14:1 EST, 25 October 2013. |. UPDATED:. 15:36 EST, 25 October 2013. The bishop of the Cargo Catholic Diocese in North Dakota has exposed potentially hundreds of church members in Cargo, Grand Works and Jamestown to the keratitis A virus in late September and early October. The state Health Department has issued an advisory of exposure for anyone who attended five churches and took communion. Bishop John Old (pictured) of the Cargo Catholic Diocese in North D...","Bishop John Old, of North Dakota, is taking time off after being diagnosed.\ne contracted the infection through contaminated food in Italy.\church members in Cargo, Grand Works and Jamestown could have been exposed.","By. Associated dress. PUBLISHED:. 14:1 EST, 25 October 2013. |. UPDATED:. 15:36 EST, 25 October 2013. The bishop of the Cargo Catholic Diocese in North Dakota has exposed potentially hundreds of church members in Cargo, Grand Works and Jamestown to the keratitis A virus in late September and early October. The state Health Department has issued an advisory against exposure to anyone who attended five churches and took communion. Bishop John Old (pictured) of the Cargo Catholic Diocese in Nor...","Bishop John Old, of North Dakota, is taking time off after being diagnosed.ne contracted the infection through contaminated food in Italy.church members in Cargo, Grand Works and Jamestown could have been exposed."
1,"(CNN) Ralph Mata was an internal affairs lieutenant for the MiamiDade Police Department, working in the division that investigates allegations of wrongdoing by cops. Outside the office, authorities allege that the 45yearold longtime officer worked with a drug trafficking organization to help plan a murder plot and get guns. A criminal complaint unsealed in U.S. District Court in New Jersey Tuesday accuses Mata, also known as ""The Milk Man,"" of using his role as a police officer to help the ...","Criminal complaint: Cop used his role to help cocaine traffickers.\nRalph Mata, an internal affairs lieutenant, allegedly helped group get guns.\nHe also arranged to pay two assassins in a murder plot, a complaint alleges.","(CNN) Ralph Data was an internal affairs lieutenant for the MiamiDade Police Department, working in the division that investigate allegation of wrongdoing by corps. Outside the office, authorities allege that the 45yearold longtime officer worked with 

In [ ]:
df_copy_correction.columns

Index(['text', 'summary', 'corrected_text', 'corrected_summary', 'punc_corrected_text', 'punc_corrected_summary', 'gram_corrected_text', 'gram_corrected_summary'], dtype='object')

In [ ]:
df_copy_correction.drop(columns=["text",
                 "summary",
                 "corrected_text",
                 "corrected_summary",
                 "punc_corrected_text",
                 "punc_corrected_summary"], inplace=True)

KeyError: ignored

In [ ]:
df_copy_correction.rename(columns={"gram_corrected_text": "text",
                   "gram_corrected_summary": "summary"}, inplace=True)

In [ ]:
df_copy_correction.drop(columns=['text', 'summary'], inplace=True)
df_copy_correction.rename(columns={"corrected_text": "text", "corrected_summary": "summary"})

,text,summary
0,"By. Associated Dress. PUBLISHED:. 14:1 EST, 25 October 2013. |. UPDATED:. 15:36 EST, 25 October 2013. The bishop of the Cargo Catholic Diocese in North Dakota has exposed potentially hundreds of church members in Cargo, Grand Works and Jamestown to the keratitis A virus in late September and early October. The state Health Department has issued an advisory of exposure for anyone who attended five churches and took communion. Bishop John Old (pictured) of the Cargo Catholic Diocese in North D...","Bishop John Old, of North Dakota, is taking time off after being diagnosed.\ne contracted the infection through contaminated food in Italy.\church members in Cargo, Grand Works and Jamestown could have been exposed."
1,"(CNN) Ralph Data was an internal affairs lieutenant for the MiamiDade Police Department, working in the division that investigate allegation of wrongdoing by corps. Outside the office, authorities allege that the 45yearold longtime officer worked with a drug trafficking organization to help plan a murder plot and get guns. A criminal complaint sealed in U.S. District Court in New Jersey Tuesday accuses Data, also known as ""The Silk An,"" of using his role as a police officer to help the drug...","Criminal complaint: Top used his role to help cocaine traffickers.\ralph Data, an internal affairs lieutenant, allegedly helped group get guns.\ne also arranged to pay two assins in a murder plot, a complaint alleged."
2,"A drunk driver who killed a young woman in a head crash while checking his mobile phone has been jailed for six years. Craig EcclestonTodd, 27, was driving home from a night at a pub when he received a text message. Is he was reading or replying to it, he veered across the road while driving round a bend and smashed into Rachel Title\he\x\x99s car coming the other way. Craig EcclestonTodd, 27 (left) was using his mobile phone when he crashed head into the car being driven by Rachel Title, 28...","Craig EcclestonTodd, 27, had drunk at least three pints before driving car.\was using phone when he veered across road in Dartmouth, Isle of Right.\thrashed head into 28yearold Rachel Title's car, who died in hospital.\notice say he would have been over legal drinkdrive limit at time of crash.\ne was found guilty at Portsmouth Grown Court of causing death by dangerous driving."
3,"(CNN) With a breezy sweep of his pen President Vladimir Ruin wrote a new chapter into Crimea's turbulent history, committing the region to a future returned to Russian domain. Sixty years prior, Ukraine's breakaway peninsula was signed away just as swiftly by Soviet leader Nikita Khrushchev. But dealing with such a latent land grab on its eastern flank will not be anywhere near as quick and easy for Europe's remember union. Because, unlike Crimea's rushed referendum, everyone has a say. Aft...","Nina do Wants says Europe must be ready to accept sanction will hurt both sides.\nTargeting Russia's business community would be one way of sapping their support for President Ruin, she says.\nut she says Europe would have a hard time keeping its factories going without power from the east."
4,"Fleetwood are the only team still to have a 100% record in Sky Met League One as a 20 win over Scunthorpe sent Graham Alexander\he\x\x99s men top of the table. The God Army are playing in the third tier for the first time in their history after six promotions in nine years and their remarkable ascent shows no sign of slowing with Amie Doctor and Earth Evans scoring the goals at Stanford Dark. Fleetwood were one of five tears to have won two out of two but the other four clubs Peterborough, B...","Fleetwood top of League One after 20 win at Scunthorpe.\nPeterborough, Bristol City, Chesterfield and Crawled all drop first points of the season.\nStandin striker Watt One scores a patrick as Rochdale trash Crewe 52.\veins for Boots County and Devil.\coventry/Bradford and Oldham/Sort Pale both end in draws.\no late Stephen Water own goal denied Gil

In [ ]:
df_copy_correction.shape

(50, 2)

In [ ]:
df_copy_correction.rename(columns={"corrected_text": "text", "corrected_summary": "summary"}, inplace=True)

In [ ]:
df_copy_correction.columns

Index(['text', 'summary'], dtype='object')

In [ ]:
df_copy_correction.isna().value_counts()

text   summary
False  False      5
dtype: int64

In [ ]:
from google.colab import files
df_copy_correction.to_csv('cnn_dailymail_cleaned.csv') 
files.download('cnn_dailymail_cleaned.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>